# Investigate Wind Direction

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import math
import numpy as np
import cartopy.crs as ccrs
import geopandas as gpd
import glob
import moviepy.editor as mpy
from palettable.colorbrewer.diverging import RdBu_11

# Load Alpine-3D netCDF file

In [2]:
# Alpine-3D grids
# ds = xr.open_dataset("../output/grids/a3d_grids.nc")
ds = xr.open_dataset("../../RL.nc")

# SNOWPACK topography 
dem = np.flipud(np.loadtxt("../input/modified_surface_grids/dem.asc", skiprows=6))
dem = xr.DataArray(dem, coords=[ds['northing'], ds['easting']], dims=['northing', 'easting'])

# Trim grids
ds = ds.isel(easting=slice(5, -5))
ds = ds.isel(northing=slice(5, -5))
dem = dem.isel(easting=slice(5, -5))
dem = dem.isel(northing=slice(5, -5))

# Plot wind speed and direction from Alpine-3D

In [3]:
'''
Calculates U and V components of wind from wind speed and direction. 
Wind direction is the meteorological wind direction (i.e. direction the wind is coming from in degrees)
'''
def calc_U_V(speed, direction):
    U = -speed * np.sin(math.pi / 180 * direction)
    V = -speed * np.cos(math.pi / 180 * direction)
    return U, V

# Get U and V components
ds['ws'] = ds['ws'].resample(time='24H').mean(dim='time')
U, V = calc_U_V(ds['ws'], ds['dw'])
U = U.resample(time='24H').mean(dim='time')
V = V.resample(time='24H').mean(dim='time')
VW_max = np.sqrt(U**2 + V**2).max()
VW_min = 0

# Calculate
# # dswe = 1000 * (ds['swe'] - ds['swe'].isel(time=0))
dswe = ds['WINDEROSIONDEPOSITION'].cumsum(dim='time').resample(time='24H').mean(dim='time')

# Get lat/lon
x_snowpack = ds['easting']
y_snowpack = ds['northing']


# # Make movie
# # Clear old images, gifs,and movies
# !mkdir -p movie_frame
# !rm -f movie.gif
# !rm -f movie.mp4
# !rm -f movie_frame/*

# for time_step in range(0, len(U['time']), 1):
for time_step in range(275, len(U['time']), 1):
    print(time_step)
    plt.clf()
    
    # Plot map of mean wind
    plt.figure(figsize=(30,15))

    # DEM
    contour_levels = np.linspace(dem.min(), dem.max(), 25)
    contour = plt.contour(x_snowpack.values, y_snowpack.values, dem, contour_levels, linestyles='solid', colors='white')
    plt.clabel(contour, fmt = '%.0f', inline = True)

    # Delta SWE
    colormap = RdBu_11.mpl_colormap # works
    plt.pcolor(x_snowpack.values, y_snowpack.values, dswe.isel(time=time_step), \
               cmap = colormap, vmin=-np.abs(dswe).max(), vmax=np.abs(dswe).max())
    plt.colorbar()

    # Wind speed and direction (still need to normalize arrows so that length means the same thing for all time steps)
    #spacing = 6
    spacing = 2
    northing_ind = np.arange(0, len(ds['northing']), spacing)
    easting_ind = np.arange(0, len(ds['easting']), spacing)
    plt.quiver(x_snowpack[easting_ind].values, y_snowpack[northing_ind].values, \
               U.isel(time=time_step)[northing_ind, easting_ind], V.isel(time=time_step)[northing_ind, easting_ind], scale=250)
    plt.title(U['time'].isel(time=time_step).values, fontsize=20)
    
    #Save Figure with image number zero padding 
    if time_step < 10: # 1 digit
        plt.savefig("movie_frame/frame_000" + str(time_step) + ".png", dpi=100)
    elif time_step < 100 and time_step > 9: # 2 digits
        plt.savefig("movie_frame/frame_00" + str(time_step) + ".png", dpi=100)
    elif time_step < 1000 and time_step > 99: # 3 digits
        plt.savefig("movie_frame/frame_0" + str(time_step) + ".png", dpi=100)
    else: # 4 digits
        plt.savefig("movie_frame/frame_" + str(time_step) + ".png", dpi=100)
    plt.close()
    
# Make a .mp4 movie and gif
file_list = sorted(glob.glob('movie_frame/*.png'))
clip = mpy.ImageSequenceClip(file_list, fps=30)
clip.write_videofile('movie.mp4')
clip.write_gif('movie.gif')


275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365


t:   1%|          | 2/366 [00:00<00:21, 16.61it/s, now=None]

Moviepy - Building video movie.mp4.
Moviepy - Writing video movie.mp4



t:   0%|          | 0/366 [00:00<?, ?it/s, now=None]          

Moviepy - Done !
Moviepy - video ready movie.mp4
MoviePy - Building file movie.gif with imageio.


<Figure size 432x288 with 0 Axes>